## Wordle Cheater 
Use Wordle Helper to find possible words (e.g., *cheat*) while you're playing Wordle. 

### Usage:
Run all cells
___



In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
import itertools
import string
import numpy as np
import requests
from lxml import html
import datetime


import ipywidgets as widgets
from IPython.display import display,display_html

import tqdm
import pdb

import pandas as pd
# from nltk.corpus import words


In [3]:
USE_LOCAL_CSV=True
if USE_LOCAL_CSV:
    df = pd.read_csv('df_wordl_words.csv')
    df_ugf = pd.read_csv('unigram_freq.csv')
else:
    sheet_url ='https://docs.google.com/spreadsheets/d/1rYkOLXEaSvq5cMY1IfpwTx4RRtIv8GNg-CyoM0Ysc84/edit#gid=2066977162'
    csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df = pd.read_csv(csv_export_url)
    sheet_url ='https://docs.google.com/spreadsheets/d/1NbUDdW_W4eTSuiRpcwnXB7B2r5JO4A3nbHfPxyIKXcY/edit#gid=1269657289'
    csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    df_ugf = pd.read_csv(csv_export_url)

df_ww = pd.read_csv('wordle_words.csv')
df_ww['word'] = df_ww.index
df_ww.index = range(len(df_ww))

df_ugf = df_ugf[~df_ugf.word.isna()]    
df_ugf.words = df_ugf.word.str.strip(' ')

df_ugf = df_ugf[df_ugf.word.isin(df_ww.word.values)]
c1 = df_ugf.word.str.len()==5
c2 = df_ugf.word.str.slice(0,1).str.lower() == df_ugf.word.str.slice(0,1)
nltk_words = df_ugf[c1 & c2].word.values
# nltk_words = df.word.values
len(nltk_words)

/var/folders/fl/tvtrn4x93cx6s2vpm9b7gn0w0000gn/T/ipykernel_1988/1325886786.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_ugf.words = df_ugf.word.str.strip(' ')


2308

In [88]:
def get_word_history():
    ret_val = None
    url_to_search = "https://screenrant.com/wordle-answers-updated-word-puzzle-guide/"
    xpath = "//a[contains(@href,'hints')]/.."
    # Make an HTTP request to the webpage
    response = requests.get(
        url_to_search, 
        headers={
            "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
        }
    )

    # Parse the HTML content
    doc = html.fromstring(response.content)

    # Execute an XPath search on the document
    results = doc.xpath(xpath)
    if len(results)>0:
        ret_val = [
            v.text_content().split('-') 
            for v in results
        ]
        ret_val = [
            [rv[0].strip(),rv[1].strip(),rv[2].strip()] 
            for rv in ret_val if len(rv)==3
        ]
        ret_val = [
            [
                rv[0],
                int(rv[1][1:]),
                rv[2]
            ]
            for rv in ret_val
        ]
        df_word_hist = pd.DataFrame(ret_val,columns=['date','number','solution'])
        df_word_hist = df_word_hist.sort_values('number',ascending=False)
        df_word_hist.index = range(len(df_word_hist))
    return df_word_hist


In [99]:
def new_condition(word,letter_status):
    good_letters = []
    for i,letter in enumerate(list(word)):
        if letter_status[i]<3:
            good_letters.append(letter)
    def inner_test(w):
        new_conditions = []
        for i,letter in enumerate(list(word)):
            if letter_status[i]==0:
                new_conditions.append(
                    letter in w 
                )                
            elif letter_status[i]==1:
                new_conditions.append(
                    letter in w[i] 
                )                
            elif letter_status[i]==2:
                new_conditions.append(
                    (letter in w) and (letter not in w[i])
                )
            else:
                new_conditions.append(
                    letter not in w
                )
        return new_conditions
    return inner_test,good_letters

class wordl():
    def __init__(self,show_progress=True):
        self.wordlist = []
        self.letter_list = []
        self.letter_status = []
        self.conditions = []
        self.show_progress = show_progress
        all_words = set(
            nltk_words
        )
        self.nltk_words =  set(
            [
                wo 
                for wo in all_words
                if (len(wo)==5) and (wo.lower()==wo)
            ]
        )
        try:
            self.df_word_history = get_word_history()
            self.df_word_history.to_csv('./temp_folder/df_word_history.csv',index=False)
        except:
            self.df_word_history = pd.read_csv('./temp_folder/df_word_history.csv')
            
            
        self.todays_word = self.df_word_history.iloc[0].solution.lower()
        
    
    def add_word(self,word,letter_status,debug_word=''):
        self.wordlist.append(word)
        self.letter_status.append(letter_status)
        condition_list,letter_list = new_condition(word,letter_status)
        self.conditions.append(condition_list)
        self.letter_list.extend(letter_list)
        self.letter_list = list(set(self.letter_list))
    
    def conds(self,w):
        cond_results = []
        for i in range(len(self.conditions)):
            cond_results.extend(self.conditions[i](w))
        return all(cond_results)
    


    def try_it(self,letter_list=None):
        words = []
        curr_lets = self.letter_list if letter_list is None else letter_list
        poss_words = [
            w for w in self.nltk_words
            if len(set(curr_lets).intersection(w))==len(curr_lets)
        ]
        for w in poss_words:
            if self.conds(w):
                words.append(w)
        return words
        
 

In [100]:
wdl = wordl()
wdl.add_word('alert',[3,3,2,3,3])
wdl.add_word('noisy',[2,3,1,3,3])
wdl.try_it()

['deign', 'eking', 'feign', 'knife', 'whine', 'being']

In [5]:
%%html
<style>
.mytext {
    text-transform:uppercase;
    font-weight: bold;
    color: blue;
    font-size:100px;
    text-align:center;
}
</style>


In [6]:
perc=100

results_output = widgets.Output(
)


In [7]:
def run_it():    
    results_output.clear_output()
    table_style = {'description_width': 'auto'}
    table_layout = {'width':'auto', 'min_width':'15px',  'min_height':'28px'}

    num_tboxes = 5
    tboxes = [
        widgets.Text(
            layout=table_layout,
    #         layout={'width':'25%'},
            style={'description_width': 'auto'} #table_style
        )
        for i in range(num_tboxes)
    ]  


    letter_status = {}
    dropdowns = []
    button_group_array = []

    def create_on_letter_button_clicked(letter_number,i):
        def on_letter_button_clicked(change):
            change.icon='check'
            letter_status[letter_number] = i
            for j in [k for k in range(3) if k != i]:
                button_group_array[letter_number][j].icon='uncheck'

        return on_letter_button_clicked

    def make_button_group(letter_number):
        button_children = []
        color_index = ["#49BE25","#D5C450","#A0A0A0"]
        for i in range(3):
            # manage icon
            icon = ''
            if i == 2:
                icon =  'check'
                letter_status[letter_number] = i
            # create button and append to button_children
            button_children.append(
                widgets.Button(style={"button_color":color_index[i]},icon=icon)
            )
            # create button observable callback for this button
            bo = create_on_letter_button_clicked(
                letter_number,i
            )
            button_children[i].on_click(bo)

        return button_children

    def make_button_groups():
        but_array = []
        for i in range(5):
            bg = make_button_group(i)
            dropdowns.append(widgets.HBox(bg))
            button_group_array.append(bg)


    make_button_groups()

    wd_box = wordl()

    def on_add_word_button_clicked(change):
        print('-----------')
        word = ''.join(
            [
                tbox.value
                for tbox in tboxes
            ]
        ).lower()
        nums = [
            letter_status[i]+1
            for i in range(5)
        ]

        print(word,nums)
        wd_box.add_word(word,nums)
        reset_word_display()

    button_add_word = widgets.Button(
        description='add_word',
        layout=widgets.Layout(width='auto', height='auto')
    )
    button_add_word.on_click(on_add_word_button_clicked)


    def on_try_it_button_clicked(change):
        words = wd_box.try_it()
        filter_words(words)
    #     display(df_ugf[df_ugf.word.isin(words)].sort_values('count',ascending=False))

    button_tryit = widgets.Button(
        description='try_it',
        layout=widgets.Layout(width='auto', height='auto')
    )
    button_tryit.on_click(on_try_it_button_clicked)

    data_entry_text = widgets.Text()
    gbox0 = widgets.GridBox(
        children=[data_entry_text],
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr',
            grid_template_columns='1fr'
        )
    )
    def gbox_observer(text_box_changed):
        if text_box_changed['name']=='value':
            if ('new' in text_box_changed) and (len(text_box_changed['new'])>0):
                for i,t in enumerate(text_box_changed['new']):
                    tboxes[i].value = t
    data_entry_text.observe(gbox_observer)

    gbox1 = widgets.GridBox(
        children=tboxes+dropdowns,
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr 1fr',
            grid_template_columns=' '.join(['auto' for _ in range(len(tboxes))])
        )
    )

    def tb_observer(text_box_changed):
        if text_box_changed['name']=='value':
            if ('new' in text_box_changed) and (len(text_box_changed['new'])>0):
                text_box_changed['owner'].value = text_box_changed['new'][-1].upper()

    for tbox in tboxes:
        tbox.add_class('mytext')
        tbox.observe(tb_observer)

    gbox2 = widgets.HBox(
        children=[button_add_word,button_tryit],
        layout=widgets.Layout(
            display='grid',
            width=f'{perc}%',
            height='auto',
            grid_template_rows='1fr',
            grid_template_columns=' '.join(['auto' for _ in range(2)])
        )
    )

    def reset_word_display():
        for i in range(5):
            tboxes[i].value = ''
            letter_status[i] = 2
            for j in range(2):
                button_group_array[i][j].icon='uncheck'
            button_group_array[i][2].icon='check'
        data_entry_text.value=''
        


    def display_df(df, gap=50, justify='center'):
        html= f"""
        <div style="display:flex; gap:{gap}px; justify-content:{justify};">
            {df._repr_html_()}
        </div>
        """
        return html
    
    def filter_words(words):
        with results_output:
            results_output.clear_output()
            df_new = df_ugf[df_ugf.word.isin(words)].sort_values('count',ascending=False).copy()
            count_sum = df_new['count'].sum()
            df_new['probability'] = df_new['count']/count_sum
            df_new.index = range(1,len(df_new)+1)
#             display(df_new)
            hh = display_df(df_new)
            display_html(hh, raw=True)        



    gbox3 = widgets.VBox([gbox0,gbox1,gbox2],width=f'{perc}%')
    
    hbox_layout = widgets.Layout(
        display='grid',
        width=f'{perc}%',
        height='auto',
        grid_template_rows='1fr',
        grid_template_columns='1fr 1fr',
    )

    display(
        widgets.HBox(
            [
                gbox3,
                results_output
            ],
            layout=hbox_layout
        )
    )


### You can play wordle cheater in the cell below

In [8]:
display(widgets.HTML('''<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> '''))
run_it()
results_output.clear_output()
# results_output

HTML(value='<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesom…

## Show occurrences of different letter combinations

In [9]:
import string
sss = string.ascii_lowercase
df_conconants = pd.DataFrame([
    [l,df_ww[df_ww.word.str.contains(l)].count().values[0]]
    for l in sss
    if l not in ['a','e','i','o','u']
],columns=['letter','num_times']).sort_values('num_times',ascending=False)

df_vowels = pd.DataFrame([
    [l,df_ww[df_ww.word.str.contains(l)].count().values[0]]
    for l in sss
    if l  in ['a','e','i','o','u']
],columns=['letter','num_times']).sort_values('num_times',ascending=False)

In [10]:
df_vowels

,letter,num_times
1,e,1053
0,a,906
3,o,672
2,i,646
4,u,456


In [11]:
df_conconants

,letter,num_times
13,r,835
15,t,667
8,l,645
14,s,617
10,n,548
1,c,446
19,y,416
5,h,377
2,d,370
11,p,345


In [12]:
def part_of(word,letters='sate'):
    return all([l in word for l in letters])

df_wwpo = df_ww[df_ww.word.apply(part_of)]
df_wwpo = df_wwpo.merge(df_ugf,on='word').sort_values('count',ascending=False)
df_wwpo['perc'] = df_wwpo['count'] / df_wwpo['count'].sum()
df_wwpo.head(20)

,word,count,perc
15,state,453104133,0.583505
6,least,111229798,0.143241
11,stage,52518330,0.067633
24,waste,41089558,0.052915
0,asset,18178753,0.023410
22,taste,17311343,0.022293
2,beast,14787561,0.019043
20,steam,11141309,0.014348
7,paste,8913055,0.011478
19,steal,6257493,0.008058


In [13]:
df_first_letters = pd.DataFrame([
    [l,df_ww[df_ww.word.str.slice(0,1)==l].count().values[0]]
    for l in sss
    if l  in string.ascii_lowercase
],columns=['letter','num_times']).sort_values('num_times',ascending=False)
df_first_letters

,letter,num_times
18,s,365
2,c,198
1,b,173
19,t,149
15,p,141
0,a,140
5,f,135
6,g,115
3,d,111
12,m,107


In [14]:
df_last_letters = pd.DataFrame([
    [l,df_ww[df_ww.word.str.slice(4,5)==l].count().values[0]]
    for l in sss
    if l  in string.ascii_lowercase
],columns=['letter','num_times']).sort_values('num_times',ascending=False)
df_last_letters

,letter,num_times
4,e,422
24,y,364
19,t,253
17,r,212
11,l,155
7,h,137
13,n,130
3,d,118
10,k,113
0,a,63


In [15]:
combos = [''.join(v) for v in list(itertools.permutations(string.ascii_lowercase,2))]
pos = [0,2]
df_common_positions = pd.DataFrame([
    [l,df_ww[df_ww.word.str.slice(pos[0],pos[1])==l].count().values[0]]
    for l in combos
],columns=['combo','num_times']).sort_values('num_times',ascending=False)
df_common_positions

,combo,num_times
468,st,65
457,sh,52
465,sp,45
66,cr,45
56,ch,40
...,...,...
305,mf,0
70,cv,0
303,md,0
302,mc,0


In [16]:
[v for v in df_ww.word.values if v[2:5] == 'ore']


['adore', 'chore', 'score', 'shore', 'snore', 'spore', 'store', 'swore']

Jan 1
